In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from scipy.integrate import solve_ivp
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm import tqdm

In [2]:
# Harmonic Oscillator Dynamics
def harmonic_oscillator(t, y, k, m):
    x, v = y
    dydt = [v, -(k / m) * x]
    return dydt

In [3]:
# Main Workflow
n_samples = 100
output_dir = "oscillator_dataset"

In [4]:
# Generate Dataset
def generate_dataset(n_samples, dt, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    dataset = []

    for i in tqdm(range(n_samples), desc="Generating dataset"):
        k = np.random.uniform(1.0, 10.0)  # Spring constant
        m = np.random.uniform(0.5, 5.0)   # Mass
        y0 = np.random.uniform(-2.0, 2.0, 2)  # Initial position and velocity
        t_span = (0, 10)
        t_eval = np.linspace(t_span[0], t_span[1], int(10 / dt))

        sol = solve_ivp(harmonic_oscillator, t_span, y0, t_eval=t_eval, args=(k, m), method='RK45')
        data = sol.y.T  # Positions and velocities over time

        gif_path = os.path.join(output_dir, f"oscillator_{i}.gif")
        generate_gif(data[:, 0], gif_path)

        dataset.append((data, k, m, gif_path))

    return dataset

In [5]:
# Generate GIFs
def generate_gif(positions, save_path):
    fig, ax = plt.subplots(figsize=(6, 2))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-0.5, 0.5)
    line, = ax.plot([], [], 'o-', lw=2)

    def update(frame):
        line.set_data([positions[frame], 0], [0, 0])
        return line,

    ani = FuncAnimation(fig, update, frames=len(positions), blit=True, interval=50)
    ani.save(save_path, fps=20, writer='imagemagick')
    plt.close(fig)

In [6]:
dataset = generate_dataset(n_samples, dt=0.01, output_dir=output_dir)   

Generating dataset: 100%|██████████| 100/100 [1:15:11<00:00, 45.12s/it]


In [7]:
# Dataset Preparation
class HarmonicOscillatorDataset(Dataset):
    def __init__(self, dataset, seq_len, pred_len):
        self.dataset = dataset
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data, k, m, gif_path = self.dataset[idx]

        gif = Image.open(gif_path)
        frames = []
        try:
            while True:
                frame = gif.convert("RGB")
                frames.append(np.array(frame))
                gif.seek(gif.tell() + 1)
        except EOFError:
            pass

        positions = data[:, 0]

        input_seq = positions[:self.seq_len]
        target_seq = positions[self.seq_len:self.seq_len + self.pred_len]

        return torch.tensor(input_seq, dtype=torch.float32), torch.tensor(target_seq, dtype=torch.float32), gif_path    

In [ ]:
seq_len = 90
pred_len = 10
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

train_dataset = HarmonicOscillatorDataset(train_data, seq_len, pred_len)
val_dataset = HarmonicOscillatorDataset(val_data, seq_len, pred_len)
test_dataset = HarmonicOscillatorDataset(test_data, seq_len, pred_len)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
input_dim = 1
hidden_dim = 64
output_dim = 1
num_layers = 2

In [ ]:
# Model Definition
class LSTMOscillatorPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMOscillatorPredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out.unsqueeze(1)

In [ ]:
# Training the Model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for inputs, targets, _ in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")

In [ ]:
# Generate Prediction GIFs
def generate_prediction_gif(model, test_loader, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model.eval()

    for i, (inputs, targets, gif_path) in enumerate(test_loader):
        with torch.no_grad():
            predictions = model(inputs).squeeze(0).numpy()

        true_positions = targets.numpy()
        input_positions = inputs.squeeze(0).numpy()

        fig, ax = plt.subplots(figsize=(6, 2))
        ax.set_xlim(-2.5, 2.5)
        ax.set_ylim(-0.5, 0.5)
        true_line, = ax.plot([], [], 'o-', lw=2, label='True Motion')
        pred_line, = ax.plot([], [], 'o-', lw=2, linestyle='--', label='Predicted Motion')

        def update(frame):
            if frame < len(input_positions):
                true_line.set_data([input_positions[frame], 0], [0, 0])
            else:
                idx = frame - len(input_positions)
                true_line.set_data([true_positions[idx], 0], [0, 0])
                pred_line.set_data([predictions[idx], 0], [0, 0])
            return true_line, pred_line

        ani = FuncAnimation(fig, update, frames=len(input_positions) + len(true_positions), blit=True, interval=50)
        ani.save(os.path.join(output_dir, f"prediction_{i}.gif"), fps=20, writer='imagemagick')
        plt.close(fig)

In [ ]:
model = LSTMOscillatorPredictor(input_dim, hidden_dim, output_dim, num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

In [ ]:
generate_prediction_gif(model, test_loader, output_dir="predicted_gifs")